In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import pyspark.sql.types as t

In [ ]:
spark = SparkSession \
    .builder \
    .appName("Spark_movies_genres_task") \
    .config("spark.master", "spark://ZHUYANZHAO-HWPC.localdomain:7077") \
    .config("spark.jars.packages", "mysql:mysql-connector-java:8.0.22") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.num", "1") \
    .getOrCreate()

In [ ]:
df_movies = spark.read.option("header","true").csv("./movies_metadata.csv")
df_movie_genres = df_movies.select(f.col("id").alias("movie_tmdb_id"), f.col("genres"))
df_movie_genres = df_movie_genres.withColumn("genres", f.from_json(f.col("genres"), "array<string>"))
df_movie_genres = df_movie_genres.withColumn("genres", f.explode("genres"))
df_movie_genres.printSchema()
df_movie_genres.show(10, truncate=False)

In [ ]:
schema = t.StructType(
    [
        t.StructField('id', t.StringType(), True),
        t.StructField('name', t.StringType(), True)
    ]
)
df_movie_genres = df_movie_genres.withColumn('genres', f.from_json('genres', schema))\
.select(f.col('movie_tmdb_id').cast('int'), f.col('genres.id').cast('int').alias('genre_id'), f.col('genres.name'))
df_movie_genres = df_movie_genres.na.drop('any')
df_movie_genres.printSchema()
df_movie_genres.show(10, truncate=False)

In [ ]:
df_movie_genres.select(*df_movie_genres.columns).write.format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/spark_movies") \
    .option("driver", "com.mysql.cj.jdbc.Driver").option("dbtable", "movie_genres") \
    .option("user", "root").option("password", "zhuyz928").save()

In [ ]:
df_genres = df_movie_genres.select(['genre_id', 'name']).distinct()
df_genres.sort(f.col('genre_id')).show(df_genres.count())

In [ ]:
df_genres.select(*df_genres.columns).write.format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/spark_movies") \
    .option("driver","com.mysql.cj.jdbc.Driver").option("dbtable", "genres") \
    .option("user", "root").option("password", "zhuyz928").save()